In [ ]:
import os
import pickle
from itertools import product
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText

In [ ]:
from tweet.tweet import Tweet
from preprocessing import preprocessing, tree_create

In [ ]:
modelft = FastText.load_fasttext_format('../model/SENTIMENT_UBA_v4_d100.bin')

In [ ]:
modelw2v = KeyedVectors.load_word2vec_format("../model/SBW-vectors-300-min5.bin.gz", binary=True)

In [ ]:
ROOT_OUT = '../pickle/'

In [ ]:
lexicon_attack = ['falso', 'mentira']

In [ ]:
# Lexicon esperado
expected_lexicon = ['mentira', 'mentiroso', 'falso', 'mentirosa', 'verdad', 'verdadero', 'engañoso', 'mentir']

In [ ]:
def taggedcross(modelft, tweets, root, lexicon, u, n):
    
    lexicons = [lexicon]
    #tree = tree_create(tweets, root)
    for i in range(n):
        new = lexicons[i]
        for _, tweet in tweets.items():
            
            merge = product(lexicons[i], tweet.tokens)

            for wordl, token in list(merge):
                if wordl and token in modelft.wv.vocab:
                    if modelft.similarity(wordl, token) >= u:
                        if token not in lexicon_attack:
                            new.append(token)
        
        lexicons.append(new)
    return lexicons

In [ ]:
tweets, root = preprocessing('perfil.status.1191780164549697536.json')

In [ ]:
lexicons = taggedcross(modelft, tweets, root, lexicon_attack, 0.65, 3)

In [ ]:
w = filter(lambda x: x in modelw2v.vocab, lexicon_attack)
lexicon_attack += [word for word, d in modelw2v.most_similar(positive=w)]